# Exporting the DataFrame to MySQL Database

## Importing the dependencies

In [ ]:
import mysql.connector as ms
from mysql.connector import MySQLConnection
import pandas as pd
import csv

## Establishing a connection with MySQL Database

In [ ]:
db = MySQLConnection(user = 'root',
                     host = 'localhost',
                     password = 'password',
                     database = 'database_name')

In [ ]:
cursor = db.cursor(buffered = True)

## Importing the DataFrame

In [ ]:
df_books = pd.read_csv('Dataset/Books.csv')
df_books.head()

In [ ]:
for i in df_books:
    print(i)

In [ ]:
for i in df_books:
    print(i, ":", sum(df_books[i].isna()))

In [ ]:
df.drop_duplicates('Book-Title', inplace = True)

In [ ]:
df_books.dropna(inplace = True)

In [ ]:
df_ratings = pd.read_csv("Dataset/Ratings.csv")

In [ ]:
df_books["ISBN"] = df_books["ISBN"].str.lower()
df_ratings["ISBN"] = df_ratings["ISBN"].str.lower()

In [ ]:
for i in df_ratings:
    print(i)

In [ ]:
df_books.rename(columns = {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'}, inplace=True)
df_ratings.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)

In [ ]:
x = df_ratings['user_id'].value_counts() > 200
y = x[x].index
print(y.shape)
df_ratings = df_ratings[df_ratings['user_id'].isin(y)]

In [ ]:
len(df_ratings)

In [ ]:
rating_with_books = df_ratings.merge(df_books, on='ISBN')
rating_with_books.head()

In [ ]:
number_rating = rating_with_books.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns= {'rating':'number_of_ratings'}, inplace=True)
final_rating = rating_with_books.merge(number_rating, on='title')
final_rating = final_rating[final_rating['number_of_ratings'] >= 25]
final_rating.drop_duplicates(['user_id','title'], inplace=True)

In [ ]:
final_rating.drop('number_of_ratings', axis = 1, inplace = True)

In [ ]:
for i in final_rating:
    print(i)

In [ ]:
df_books = final_rating[['ISBN', 'title', 'author', 'year', 'publisher',
                         'Image-URL-S', 'Image-URL-M', 'Image-URL-L']]

df_ratings = final_rating[['user_id', 'ISBN', 'rating']]

In [ ]:
df_books.shape

In [ ]:
df_ratings.shape

In [ ]:
df_books.drop_duplicates(inplace = True)
df_books.shape

In [ ]:
for i in df_books:
    print(i)

## Creating the Table

In [ ]:
books = """
CREATE TABLE books(ISBN VARCHAR(255) NOT NULL,
Title VARCHAR(255) NOT NULL,
Author VARCHAR(255) NOT NULL,
Year INT(4) NOT NULL,
Publisher VARCHAR(255) NOT NULL,
Image_URL_S VARCHAR(255) NOT NULL,
Image_URL_M VARCHAR(255) NOT NULL,
Image_URL_L VARCHAR(255) NOT NULL);
"""

In [ ]:
cursor.execute(books)

In [ ]:
cursor.execute("SHOW TABLES;")
cursor.fetchall()

## Adding Values to the Tables

In [ ]:
book_values = df_books.values

In [ ]:
for i in book_values:
    print(i)

In [ ]:
for i in range(0, len(book_values)):
    cursor.execute("""
    INSERT INTO books(ISBN, Title, Author, Year, Publisher, Image_URL_S, Image_URL_M, Image_URL_L)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s)
    """,list(book_values[i]))
    print(i)

## Adding the rest of the tables to the database

In [ ]:
df_ratings['user_id'].value_counts()

In [ ]:
df_ratings.head()

In [ ]:
cursor.execute('''CREATE TABLE ratings(User_ID VARCHAR(255),
ISBN VARCHAR(255),
Book_Rating int(2));''')

In [ ]:
cursor.execute("SHOW TABLES;")
cursor.fetchall()

In [ ]:
ratings = df_ratings.values

In [ ]:
for i in range(0, len(ratings)):
    cursor.execute("""
    INSERT INTO ratings(USER_ID, ISBN, Book_Rating)
    VALUES(%s,%s,%s)
    """,list(ratings[i]))
    print(i)

In [ ]:
cursor.execute("SELECT * FROM ratings;")

In [ ]:
t = (cursor.fetchall())

In [ ]:
t